In [2]:
%pip install google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.Collecting google-auth-oauthlib
  Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.3.1-py3-none-any.whl.metadata (7.9 kB)
Using cached google_auth_oauthlib-1.2.2-py3-none-any.whl (19 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached oauthlib-3.3.1-py3-none-any.whl (160 kB)




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
import email

# Scopes for reading Gmail messages
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=8080)
    service = build('gmail', 'v1', credentials=creds)
    return service

def get_messages(service, query='', max_results=100):
    result = service.users().messages().list(userId='me', q=query, maxResults=max_results).execute()
    messages = result.get('messages', [])
    return messages

def get_message_content(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='raw').execute()
    raw_data = base64.urlsafe_b64decode(msg['raw'].encode('ASCII'))
    mime_msg = email.message_from_bytes(raw_data)
    body = ''
    if mime_msg.is_multipart():
        for part in mime_msg.walk():
            if part.get_content_type() == 'text/plain':
                body += part.get_payload(decode=True).decode('utf-8', errors='ignore') #type: ignore
    else:
        body = mime_msg.get_payload(decode=True).decode('utf-8', errors='ignore') #type: ignore
    return body

def get_message_subject(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='metadata').execute()
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'] == 'Subject':
            return header['value']
    return None

def get_message_sender(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='metadata').execute()
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'] == 'From':
            return header['value']
    return None

def get_message_date(service, msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='metadata').execute()
    headers = msg['payload']['headers']
    for header in headers:
        if header['name'] == 'Date':
            return header['value']
    return None

In [2]:
service = authenticate_gmail()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=628982370333-kkqj66e4850ij5qm0nr2l4b6a514s23e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=KkTDpnl6HqwXQ02zdQHOkYhdpuOBkN&access_type=offline


In [3]:
spam = get_messages(service, query='label:SPAM')
ham = get_messages(service, query='category:primary -label:SPAM')
print(f"Found {len(spam)} spam messages and {len(ham)} ham messages.")

Found 17 spam messages and 100 ham messages.


In [12]:
spam_content = {}
for msg in spam:
    msg_id = msg['id']
    subject = get_message_subject(service, msg_id)
    sender = get_message_sender(service, msg_id)
    date = get_message_date(service, msg_id)
    content = get_message_content(service, msg_id)
    
    spam_content[msg_id] = {
        'subject': subject,
        'sender': sender,
        'date': date,
        'content': content
    }

In [16]:
ham_content = {}
for msg in ham:
    msg_id = msg['id']
    subject = get_message_subject(service, msg_id)
    sender = get_message_sender(service, msg_id)
    date = get_message_date(service, msg_id)
    content = get_message_content(service, msg_id)
    
    ham_content[msg_id] = {
        'subject': subject,
        'sender': sender,
        'date': date,
        'content': content
    }

In [17]:
#save the spam and ham content as JSON files
import json
with open('spam_content.json', 'w') as f:
    json.dump(spam_content, f, indent=4)
with open('ham_content.json', 'w') as f:
    json.dump(ham_content, f, indent=4)
print("Spam and ham content saved to 'spam_content.json' and 'ham_content.json'.")

Spam and ham content saved to 'spam_content.json' and 'ham_content.json'.


In [18]:
from gmail_api import fetch_spam_and_ham
# Example usage of the fetch_spam_and_ham function
spam, ham = fetch_spam_and_ham(100)
print(f"Fetched {len(spam)} spam messages and {len(ham)} ham messages using the fetch_spam_and_ham function.")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=628982370333-kkqj66e4850ij5qm0nr2l4b6a514s23e.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=wyyM7IaRx0uigIkmRoIz68jMMYeGVm&access_type=offline
Found 17 spam messages and 100 ham messages.
Fetched 17 spam messages and 100 ham messages using the fetch_spam_and_ham function.
